# 教程4: Motif活性分数计算

在这里，我们使用`scbasset`对scATAC里染色质开放区域的Motif进行活性评估，目的是在于找到转录因子的结合基序的位置的分数

详细教程见https://github.com/calico/scBasset


In [1]:
#导入包
import anndata
print('anndata(Ver): ',anndata.__version__)
import scanpy as sc
print('scanpy(Ver): ',sc.__version__)
import matplotlib.pyplot as plt
import matplotlib
print('matplotlib(Ver): ',matplotlib.__version__)
import seaborn as sns
print('seaborn(Ver): ',sns.__version__)
import numpy as np
print('numpy(Ver): ',np.__version__)
import pandas as pd
print('pandas(Ver): ',pd.__version__)
import scvelo as scv
print('scvelo(Ver): ',scv.__version__)
import Pyomic
print('Pyomic(Ver): ',Pyomic.__version__)
import scvi
print('scvi(Ver): ',scvi.__version__)
import scglue
print('scglue(Ver): ',scglue.__version__)


#绘图参数设置
sc.settings.verbosity = 3             # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.settings.set_figure_params(dpi=80, facecolor='white')

from matplotlib.colors import LinearSegmentedColormap
sc_color=['#7CBB5F','#368650','#A499CC','#5E4D9A','#78C2ED','#866017', '#9F987F','#E0DFED',
 '#EF7B77', '#279AD7','#F0EEF0', '#1F577B', '#A56BA7', '#E0A7C8', '#E069A6', '#941456', '#FCBC10',
 '#EAEFC5', '#01A0A7', '#75C8CC', '#F0D7BC', '#D5B26C', '#D5DA48', '#B6B812', '#9DC3C3', '#A89C92', '#FEE00C', '#FEF2A1']
sc_color_cmap = LinearSegmentedColormap.from_list('Custom', sc_color, len(sc_color))

anndata(Ver):  0.8.0
scanpy(Ver):  1.9.1
matplotlib(Ver):  3.5.1
seaborn(Ver):  0.11.2
numpy(Ver):  1.22.3
pandas(Ver):  1.3.5
scvelo(Ver):  0.2.4
Pyomic(Ver):  1.1.4


Global seed set to 0


scvi(Ver):  0.20.1
scglue(Ver):  0.3.2


### 加载数据和预处理
在本次教程, 我们将使用`scglue`的atac-emb文件作为输入

In [2]:
current_path='/home/leihu/data/analysis/rb_tutorial/'
adata=sc.read(current_path+'data/scglue/atac-emb.h5ad')
adata

AnnData object with n_obs × n_vars = 21194 × 296334
    obs: 'Tissue', 'Developmental_Stage', 'SRR', 'nb_features', 'log_nb_features', 'domain', 'balancing_weight'
    var: 'n_cells', 'commonness', 'prop_shared_cells', 'variability_score', 'chrom', 'chromStart', 'chromEnd', 'highly_variable'
    uns: '__scglue__', 'neighbors', 'umap'
    obsm: 'X_glue', 'X_lsi', 'X_umap'
    varm: 'X_glue'
    obsp: 'connectivities', 'distances'

We can use scanpy functions to handle, filter, and manipulate the data. In our case, we might want to filter out peaks that are rarely detected, to make the model train faster:

In [3]:
print(adata.shape)
# compute the threshold: 5% of the cells
min_cells = int(adata.shape[0] * 0.05)
# in-place filtering of regions
sc.pp.filter_genes(adata, min_cells=min_cells)
print(adata.shape)

(21194, 296334)
filtered out 260884 genes that are detected in less than 1059 cells
(21194, 35450)


我们需要将类似chr10_100173036_100173942分割成chr10，100173036，100173942三个部分

In [4]:
adata.var['gene_ids']=adata.var.index.tolist()
split_interval = adata.var["gene_ids"].str.split("_", expand=True)
adata.var["chr"] = split_interval[0]
adata.var["start"] = split_interval[1].astype(int)
adata.var["end"] = split_interval[2].astype(int)
adata.var

,n_cells,commonness,prop_shared_cells,variability_score,chrom,chromStart,chromEnd,highly_variable,gene_ids,chr,start,end
name,,,,,,,,,,,,
chr10_100173036_100173942,1498,4603.0,0.114983,0.614983,chr10,100173036,100173942,False,chr10_100173036_100173942,chr10,100173036,100173942
chr10_100256039_100256897,2862,7228.0,0.180556,0.680556,chr10,100256039,100256897,False,chr10_100256039_100256897,chr10,100256039,100256897
chr10_100312289_100313212,3346,8659.0,0.216302,0.716302,chr10,100312289,100313212,False,chr10_100312289_100313212,chr10,100312289,100313212
chr10_100403022_100404021,1760,6837.0,0.170788,0.670788,chr10,100403022,100404021,False,chr10_100403022_100404021,chr10,100403022,100404021
chr10_100414398_100415297,3531,8736.0,0.218225,0.718225,chr10,100414398,100415297,False,chr10_100414398_100415297,chr10,100414398,100415297
...,...,...,...,...,...,...,...,...,...,...,...,...
chrY_19975993_19976805,2816,6709.0,0.167591,0.667591,chrY,19975993,19976805,False,chrY_19975993_19976805,chrY,19975993,19976805
chrY_20723229_20724038,2306,5670.0,0.141637,0.641637,chrY,20723229,20724038,False,chrY_20723229_20724038,chrY,20723229,20724038
chrY_2518055_2519065,1615,3957.0,0.098846,0.598846,chrY,2518055,2519065,False,chrY_2518055_2519065,chrY,2518055,2519065


此外，有一些染色体的开头不是chr，我们在分析中也应该去掉

In [5]:
# Filter out non-chromosomal regions
mask = adata.var["chr"].str.startswith("chr")
adata = adata[:, mask].copy()

In [8]:
adata.write_h5ad(current_path+'data/scbasset/adata_scbasset_pre.h5ad',compression='gzip')

### scBasset模型预处理与训练

模型的训练参数参照[scbasset](https://github.com/calico/scBasset/)的官方文档，我这里简单解释一下参数

预处理：scbasset_preprocess.py
- ad_file 输入的scATAC-seq数据文件，其染色体已被分割，peaks已被筛选
- input_fasta 输入的参考基因组，在这里由于我们是用T2TCHM13对测序数据进行比对的，所以我们参考基因组也选择CHM13.fa
- out_path 模型预处理好后的保存路径

训练：scbasset_train.py
- input_folder 模型预处理的保存路径
- epochs 训练迭代次数，其实经过我多次摸索，200个epochs模型就能收敛地比较好了
- out_path 模型训练好后的保存路径

In [ ]:
!python /home/leihu/data/analysis/rb_tutorial/scBasset/bin/scbasset_preprocess.py \
--ad_file /home/leihu/data/analysis/rb_tutorial/data/scbasset/atac_scbasset_pre.h5ad \
--input_fasta /home/leihu/data/Reference/CHM13/fasta/genome.fa \
--out_path /home/leihu/data/analysis/rb_tutorial/model/scbasset/processed


In [ ]:
!python /home/leihu/data/analysis/rb_tutorial/scBasset/bin/scbasset_train.py \
--input_folder /home/leihu/data/analysis/rb_tutorial/model/scbasset/processed \
--epochs 1000 \
--out_path /home/leihu/data/analysis/rb_tutorial/model/scbasset

### 模型加载

在这里，我们提供了一个训练好的模型存放在`'model/scbasset/best_model.h5'`路径下

In [6]:
# load model
from scbasset.utils import *

model = make_model(32, adata.shape[0], show_summary=False)
model.load_weights(current_path+'model/scbasset/best_model.h5')

2023-03-11 01:12:40.039812: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-03-11 01:12:40.039877: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-03-11 01:12:40.039883: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


2023-03-11 01:12:40.420707: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-03-11 01:12:40.420723: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)


### Motif活性评估

在这里，我们提供两种方法对Motif进行活性评估，一种是作者给的方法，由于该方法最开始设计的时候是为了评估一个转录因子的Motif，所以如果想处理所有的转录因子效率就会比较低，因为涉及两个重复的步骤：`模型构建`与`背景motif评估`，于是我将这两个步骤给单独抽了出来，实现了第二种方法

In [7]:
motif_list=[i.split('.fasta')[0] for i in os.listdir(current_path+'cis_reg/Homo_sapiens_motif_fasta/shuffled_peaks_motifs')]
motif_list[:5]

['MBD2', 'RFX7', 'ZNF282', 'OLIG3', 'HOXC6']

#### 方法1:

我们使用作者给的`motif_score`函数对motif进行评分

In [ ]:
#为每一个motif进行评分，一共是733个，大概也要算个几小时才能评估完
#k是用来记录当前到第几个了
#丑陋的写法，scbasset的作者
import gc
k=0
new_pd=pd.DataFrame(columns=adata.obs.index.tolist())
for i in motif_list:
    print(k,i)
    scores = motif_score(i, model, motif_fasta_folder=current_path+'cis_reg/Homo_sapiens_motif_fasta/')
    new_pd.loc[i] = scores
    k+=1
    gc.collect()


#### 方法2:

我们直接使用模型来批量预测TF的motif分数

In [9]:
#模型构建与参数筛选
new_model = tf.keras.Model(
            inputs=model.layers[0].input,
            outputs=model.layers[-4].output,
        )
w = model.layers[-3].get_weights()[0]

In [11]:
#背景motif计算
records = list(SeqIO.parse('/home/leihu/data/analysis/rb_tutorial/cis_reg/Homo_sapiens_motif_fasta/shuffled_peaks.fasta', "fasta"))
seqs = [str(i.seq) for i in records]
seqs_1hot = np.array([dna_1hot(i) for i in seqs])
Y_pred = new_model.predict(seqs_1hot)
accessibility_norm = np.dot(Y_pred.squeeze(), w)
pred_bg = accessibility_norm = np.dot(Y_pred.squeeze(), w)

32/32 [==============================] - 2s 70ms/step


In [ ]:
#优雅的写法
import gc
new_pd=pd.DataFrame(columns=adata.obs.index.tolist())
for i,k in zip(motif_list,range(len(motif_list))):
    print(k,i)
    #计算要预测的转录因子的motif
    records = list(SeqIO.parse('/home/leihu/data/analysis/rb_tutorial/cis_reg/Homo_sapiens_motif_fasta/shuffled_peaks_motifs/{}.fasta'.format(i), "fasta"))
    seqs = [str(i.seq) for i in records]
    seqs_1hot = np.array([dna_1hot(i) for i in seqs])
    Y_pred = new_model.predict(seqs_1hot)
    accessibility_norm = np.dot(Y_pred.squeeze(), w)
    pred_motif = accessibility_norm = np.dot(Y_pred.squeeze(), w)
    #预测的减去背景
    tf_score = pred_motif.mean(axis=0) - pred_bg.mean(axis=0)
    tf_score = (tf_score - tf_score.mean()) / tf_score.std()
    new_pd.loc[i] = tf_score
    gc.collect()

### 保存文件

In [ ]:
#保存文件
new_pd.to_csv(current_path+'data/scbasset/atac_motif_act_80.csv')

In [ ]:
#我们还可以按照anndata的格式保存文件
adata_act=anndata.AnnData(new_pd.T,obs=pd.DataFrame(index=new_pd.columns.tolist()),
                          var=pd.DataFrame(index=new_pd.index.tolist()))
adata_act

In [ ]:
adata_act.write_h5ad(current_path+'data/scbasset/atac_motif_act.h5ad',compression='gzip')